In [19]:
import os
import qiime2.plugins.dada2.actions as dada2_actions
import qiime2.plugins.demux.actions as demux_actions
import qiime2.plugins.diversity.actions as diversity_actions
import qiime2.plugins.emperor.actions as emperor_actions
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions
import qiime2.plugins.feature_table.actions as feature_table_actions
import qiime2.plugins.metadata.actions as metadata_actions
import qiime2.plugins.phylogeny.actions as phylogeny_actions
import qiime2.plugins.taxa.actions as taxa_actions
import zipfile

from qiime2 import Metadata, Artifact
from urllib import request
from pathlib import Path

In [20]:
# Download the metadata and save it to both a file and an object

url = 'https://docs.qiime2.org/2024.5/data/tutorials/moving-pictures-usage/sample-metadata.tsv'
fn = 'sample-metadata.tsv'
request.urlretrieve(url, fn)
sample_metadata_md = Metadata.load(fn)

In [21]:
# Download the zipped sequences and unzip them, delete the zipped file

url = 'https://docs.qiime2.org/2024.5/data/tutorials/moving-pictures-usage/emp-single-end-sequences.zip'
file_path = 'emp-single-end-sequences.zip'
request.urlretrieve(url, file_path)
with zipfile.ZipFile(file_path) as zipped:
    zipped.extractall('emp-single-end-sequences')
os.remove(file_path)

In [22]:
# Load the file into an artifact

emp_single_end_sequences  = Artifact.import_data(
    type = 'EMPSingleEndSequences',
    view = 'emp-single-end-sequences')

emp_single_end_sequences.uuid, emp_single_end_sequences.format, emp_single_end_sequences.format

(UUID('f591564b-d48e-49e1-9494-1dd57d44c173'),
 q2_types.multiplexed_sequences._format.EMPSingleEndDirFmt,
 q2_types.multiplexed_sequences._format.EMPSingleEndDirFmt)

In [23]:
# Demultiplex the sequences using the barcodes from the provided metadata file, save visualized results

barcode_seq_mdc = sample_metadata_md.get_column('barcode-sequence')
demux, demux_details = demux_actions.emp_single(
    seqs = emp_single_end_sequences,
    barcodes = barcode_seq_mdc
)
demux_viz = demux_actions.summarize(data = demux)
demux_viz.visualization.save('demux_viz.qzv')

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_demux/_summarize/_visualizer.py:180: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  context['result_data'] = pd.concat([context['result_data'], df])
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_demux/_summarize/_visualizer.py:74: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['152 n

'demux_viz.qzv'

In [24]:
# Create feature table via dada2 denoising
# trim_left and trunc_len selected via demux.qzv quality plots

table, rep_seqs, stats = dada2_actions.denoise_single(
    demultiplexed_seqs=demux,
    trim_left=0,
    trunc_len=120,
)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /tmp/qiime2/root/data/d15f9d5f-3ee5-411c-8fe4-a260e165924e/data --output_path /tmp/tmpefojc8g1/output.tsv.biom --output_track /tmp/tmpefojc8g1/track.tsv --filtered_directory /tmp/tmpefojc8g1 --truncation_length 120 --trim_left 0 --max_expected_errors 2.0 --truncation_quality_score 2 --max_length Inf --pooling_method independent --chimera_method consensus --min_parental_fold 1.0 --allow_one_off False --num_threads 1 --learn_min_reads 1000000 --homopolymer_gap_penalty NULL --band_size 16

R version 4.3.3 (2024-02-29) 


Loading required package: Rcpp


DADA2: 1.30.0 / Rcpp: 1.0.12 / RcppParallel: 5.1.6 
2) Filtering ..................................
3) Learning Error Rates
19539480 total bases in 162829 reads from 34 samples will be used for learning the error rates.
4) Denoise samples 
..................................
5) Remove chimeras (method = consensus)
6) Report read numbers through the pipeline
7) Write output


In [25]:
# Visualize dada2 stats

stats_dada2_md_md = stats.view(Metadata)
stats_viz, = metadata_actions.tabulate(
    input=stats_dada2_md_md,
)
stats_viz.save('stats')

'stats.qzv'

In [26]:
# Visualize feature table summary
table_viz, = feature_table_actions.summarize(
    table=table,
    sample_metadata=sample_metadata_md,
)
table_viz.save('table_summary')

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


'table_summary.qzv'

In [27]:
# Visualize sequence tabulation
rep_seqs_viz, = feature_table_actions.tabulate_seqs(
    data=rep_seqs,
)
rep_seqs_viz.save('rep-seqs')

'rep-seqs.qzv'

In [28]:
# Create phylogenetic tree via mafft 

action_results = phylogeny_actions.align_to_tree_mafft_fasttree(
    sequences=rep_seqs,
)
aligned_rep_seqs = action_results.alignment
masked_aligned_rep_seqs = action_results.masked_alignment
unrooted_tree = action_results.tree
rooted_tree = action_results.rooted_tree

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2/root/data/5ac36e29-c1d2-4c89-b696-1ea0912e331d/data/dna-sequences.fasta



inputfile = orig
770 x 120 - 120 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  701 / 770 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
  760 / 770
done.

Progressive alignment 1/2... 
STEP   347 / 769 (thread    0)
Reallocating..done. *alloclen = 1241
STEP   701 / 769 (thread    0) h
done.

Making a distance matrix from msa.. 
  700 / 770 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
  760 / 770
done.

Progressive alignment 2/2... 
STEP   601 / 769 (thread    0)
Reallocating..done. *alloclen = 1241
STEP   701 / 769 (thread    0)
done.

disttbfast (nuc) Version 7.520
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto in

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2/root/data/2b3ba90d-d785-41e1-80e5-169694fee144/data/aligned-dna-sequences.fasta



FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /tmp/qiime2/root/data/2b3ba90d-d785-41e1-80e5-169694fee144/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
      0.10 seconds: Joined    700 of    750
Initial topology in 0.11 seconds
Refining topology: 38 rounds ME-NNIs, 2 rounds ME-SPRs, 19 rounds ML-NNIs
      0.20 seconds: SPR round   1 of   2, 701 of 1504 nodes
      0.30 seconds: SPR round   2 of   2, 1 of 1504 nodes
      0.41 seconds: SPR round   2 of   2, 1001 of 1504 nodes
Total branch-length 39.069 after 0.48 sec
      0.53 seconds: ML NNI round 1 of 19, 101 of 751 splits, 14 changes (max delta 3.606)
      0.63 seconds: ML NNI round 1 of 19, 601 of 751 splits, 109 changes (max delta 9.502)
ML-NNI round 1: LogLk = -23827.725 NNIs 136

In [29]:
# Generate diversity analysis, save artifacts we use later

action_results = diversity_actions.core_metrics_phylogenetic(
    phylogeny=rooted_tree,
    table=table,
    sampling_depth=1103,
    metadata=sample_metadata_md,
)
bray_curtis_pcoa_results = action_results.bray_curtis_pcoa_results
evenness_vector = action_results.evenness_vector
faith_pd_vector = action_results.faith_pd_vector
unweighted_unifrac_distance_matrix = action_results.unweighted_unifrac_distance_matrix
unweighted_unifrac_pcoa_results = action_results.unweighted_unifrac_pcoa_results

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.011379263397379774 and the largest is 3.5988447658955054.
  warn(


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/root/data/f25c1c03-9a6c-4eb1-bf3b-b1993243f3bb/data/feature-table.biom -t /tmp/qiime2/root/data/1386054d-59c8-41db-8c84-0fbe0268d6a5/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-ue2b9o2q

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/root/data/f25c1c03-9a6c-4eb1-bf3b-b1993243f3bb/data/feature-table.biom -t /tmp/qiime2/root/data/1386054d-59c8-41db-8c84-0fbe0268d6a5/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-j2mv5mlj

Running external command line application. This may print messages to stdout and/or stderr.
The command being

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.012830228907581964 and the largest is 2.7245904946800072.
  warn(
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more d

In [30]:
# Save richness and evenness visualizations

faith_pd_group_significance_viz, = diversity_actions.alpha_group_significance(
    alpha_diversity=faith_pd_vector,
    metadata=sample_metadata_md,
)
evenness_group_significance_viz, = diversity_actions.alpha_group_significance(
    alpha_diversity=evenness_vector,
    metadata=sample_metadata_md,
)
faith_pd_group_significance_viz.save('faith_viz')
evenness_group_significance_viz.save('evenness.viz')

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


'evenness.viz.qzv'

In [31]:
# Create and save visualizations for significant relationships between categorical metadata and beta-diversity
body_site_mdc = sample_metadata_md.get_column('body-site')
unweighted_unifrac_body_site_group_significance_viz, = diversity_actions.beta_group_significance(
    distance_matrix=unweighted_unifrac_distance_matrix,
    metadata=body_site_mdc,
    pairwise=True,
)
unweighted_unifrac_body_site_group_significance_viz.save('body-site')
subject_mdc = sample_metadata_md.get_column('subject')
unweighted_unifrac_subject_group_significance_viz, = diversity_actions.beta_group_significance(
    distance_matrix=unweighted_unifrac_distance_matrix,
    metadata=subject_mdc,
    pairwise=True,
)
unweighted_unifrac_body_site_group_significance_viz.save('subject')

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_diversity/_beta/_visualizer.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pairs_summary = pd.concat([pairs_summary, group_pairs_summary])
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_diversity/_beta/_visualizer.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pairs_summary = pd.concat([pairs_summary, group_pairs_summary])


'subject.qzv'

In [32]:
# Generate principle coordinate analysis over time using days since experiment start

unweighted_unifrac_emperor_days_since_experiment_start_viz, = emperor_actions.plot(
    pcoa=unweighted_unifrac_pcoa_results,
    metadata=sample_metadata_md,
    custom_axes=['days-since-experiment-start'],
)
unweighted_unifrac_emperor_days_since_experiment_start_viz.save('unweighted_unifrac_emperor_days_since_experiment_start_viz')
bray_curtis_emperor_days_since_experiment_start_viz, = emperor_actions.plot(
    pcoa=bray_curtis_pcoa_results,
    metadata=sample_metadata_md,
    custom_axes=['days-since-experiment-start'],
)
bray_curtis_emperor_days_since_experiment_start_viz.save('bray_curtis_emperor_days_since_experiment_start_viz')

'bray_curtis_emperor_days_since_experiment_start_viz.qzv'

In [33]:
alpha_rarefaction_viz, = diversity_actions.alpha_rarefaction(
    table=table,
    phylogeny=rooted_tree,
    max_depth=4000,
    metadata=sample_metadata_md,
)

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/root/data/d22ca145-9aea-4902-9edd-eab56ae88cc0/data/feature-table.biom -t /tmp/qiime2/root/data/0b07ed43-dbab-413c-b06b-b9846065d513/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-hbewgy7v



/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/root/data/4d08aade-6e20-4e5b-a0a3-4b6237344e6f/data/feature-table.biom -t /tmp/qiime2/root/data/0b07ed43-dbab-413c-b06b-b9846065d513/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-xrcjlcde



/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


...

/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/root/data/8f9a22a1-e5c4-4e78-95e3-45d71b53ce64/data/feature-table.biom -t /tmp/qiime2/root/data/0b07ed43-dbab-413c-b06b-b9846065d513/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-v376_jl8



/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/root/data/f66f73ac-e5bd-43c2-8480-d956d22d9e64/data/feature-table.biom -t /tmp/qiime2/root/data/0b07ed43-dbab-413c-b06b-b9846065d513/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-yb328g11



/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_types/sample_data/_transformer.py:27: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/opt/conda/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_diversity/_alpha/_visualizer.py:260: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and s

In [34]:
url = 'https://docs.qiime2.org/2024.5/data/tutorials/moving-pictures-usage/gg-13-8-99-515-806-nb-classifier.qza'
fn = 'gg-13-8-99-515-806-nb-classifier.qza'
request.urlretrieve(url, fn)
gg_13_8_99_515_806_nb_classifier = Artifact.load(fn)

In [35]:
# Use naive bayes classifier for taxonomy assignment

taxonomy, = feature_classifier_actions.classify_sklearn(
    classifier=gg_13_8_99_515_806_nb_classifier,
    reads=rep_seqs,
)
taxonomy_as_md_md = taxonomy.view(Metadata)
taxonomy_viz, = metadata_actions.tabulate(
    input=taxonomy_as_md_md,
)
taxonomy_viz.save('taxonomy')

'taxonomy.qzv'

In [36]:
# Create taxonomic bar plot

taxa_bar_plots_viz, = taxa_actions.barplot(
    table=table,
    taxonomy=taxonomy,
    metadata=sample_metadata_md,
)
taxa_bar_plots_viz.save('taxonomy_bar')

'taxonomy_bar.qzv'

In [41]:
# Clean up vizualizations
cwd = Path(os.getcwd())
os.mkdir(cwd.joinpath('vizualizations'))
for file in os.listdir(cwd):
    if file[-4:] == '.qzv':
        os.rename(cwd.joinpath(file), cwd.joinpath('vizualizations').joinpath(file))